# A NEAT Demonstration

NEAT provides a way to define graph machine learning tasks with minimal coding, an uncomplicated interface, and a process created with cloud compute in mind.

This notebook provides a demonstration of how to set up a NEAT configuration file. We define the values for parameters, write them to a YAML file, then pass that file to NEAT to generate graph embeddings.

You're likely reading this notebook while within the NEAT repository. If you haven't installed NEAT yet, please do so now using the next code block.

In [ ]:
%cd ..
%pip install .
%cd notebooks/

## Define graph parameters

For demonstration purposes, we'll use a copy of the [ECTO ontology](https://obofoundry.org/ontology/ecto.html), pre-processed to graph form by [KG-OBO](https://github.com/Knowledge-Graph-Hub/kg-obo).

In [ ]:
!wget https://kg-hub.berkeleybop.io/kg-obo/ecto/2022-03-09/ecto_kgx_tsv.tar.gz
!tar xvzf ecto_kgx_tsv.tar.gz

Now define the following graph parameters or just use the default values.

In [ ]:
directed = False # Yes, this is technically a directed network, but we'll treat it as undirected
node_path = "ecto_kgx_tsv_nodes.tsv" # Our positive training nodes
edge_path = "ecto_kgx_tsv_edges.tsv" # Our positive training edges - note that this will change shortly

We want to have a positive validation set, too. Let's generate a ~20% holdout set based on the edge file just downloaded.

In [ ]:
import random

train_edge_path = (edge_path.split("."))[0] + "_train.tsv"
pos_valid_edge_path = (edge_path.split("."))[0] + "_validate.tsv"

with open(edge_path, 'r') as edge_file:
    header = edge_file.readline()
    with open(train_edge_path, 'w') as train_edge_file:
        train_edge_file.write(header)
        with open (pos_valid_edge_path, 'w') as valid_edge_file:
            valid_edge_file.write(header)
            for line in edge_file:
                if random.randint(1,100) < 20:
                    valid_edge_file.write(line)
                else:
                    train_edge_file.write(line)
                    
edge_path = train_edge_path

Negative data would be nice to have, too. We'll define paths for negative training and negative validation edge lists here, but won't create them until the next section.

In [ ]:
neg_train_edge_path = "negative_edges.tsv"
neg_valid_edge_path = "negative_valid_edges.tsv"

## Loading a graph and generating subgraphs

First task: load our training nodes and edges with Ensmallen and get some details about the graph.

In [ ]:
from ensmallen import Graph

In [ ]:
g= Graph.from_csv(
  directed=directed,
  node_path=node_path,
  edge_path=edge_path,
  verbose=True,
  nodes_column='id',
  node_list_node_types_column='category',
  default_node_type='biolink:NamedThing',
  sources_column='subject',
  destinations_column='object',
  edge_list_edge_types_column='predicate'
)
g

Now let's generate some negative edge sets. The following method will create a new Graph object with the specified number of edges, where each is a negative edge in the original graph. It doesn't impact nodes at all, so we remove the now-disconnected nodes for the sake of cleanliness.

In [ ]:
negative_graph = g.sample_negatives(g.get_edges_number()) # Just as many negative examples as positive examples
negative_graph = negative_graph.drop_disconnected_nodes()
negative_graph

Now we'll get Ensmallen to generate two different holdout graphs for us, write out those edge files, and continue with defining the NEAT configuration.

In [ ]:
neg_train_edge_graph, neg_valid_edge_graph = negative_graph.random_holdout(train_size=0.8)
neg_train_edge_graph.dump_edges(neg_train_edge_path, edges_type_column='predicate')
neg_valid_edge_graph.dump_edges(neg_valid_edge_path, edges_type_column='predicate')

## Define embedding parameters

These parameters are quite simple for purposes of the demonstration.

In [ ]:
embedding_file_name = "demo_embeddings.tsv"
embedding_history_file_name = "embedding_history.json"
node_embedding_method_name = "CBOW" # one of 'CBOW', 'GloVe', 'SkipGram', 'Siamese', 'TransE', 'SimplE', 'TransH', 'TransR'
walk_length = 10 # typically 100 or so
batch_size = 128 # typically 512? or more
window_size = 4
iterations = 5 # typically 20 or more

## Define classifier parameters

Here, we define a single classifier, but NEAT will accept a list of multiple classifier types.

We're going to set it up as if we were building a link prediction model. NEAT would prefer to have positive *and* negative training/validation data for this. Conveniently, we've provided paths to those above. 

We may specify an exact set of edge types to predict, e.g., `biolink:subclass_of`, though here we consider all edge types.

In [ ]:
edge_method = "Average" # one of EdgeTransformer.methods: Hadamard, Sum, Average, L1, AbsoluteL1, L2, or alternatively a lambda
classifier_type = "Logistic Regression"
classifier_model_outfile = "model_lr_demo"
classifier_model_type = "sklearn.linear_model.LogisticRegression"
classifier_model_random_state = 42
classifier_model_max_iter = 1000

## Define output parameters

We specify a local output path here, but NEAT can also upload to S3, given a bucket name and directory.

In [ ]:
output_directory = "./"

config_filename = "demonstrate.yaml"

## Wrap it all up

One last detail before assembling the config file: we would like to categorize our nodes, but the existing categories are rather homogeneous.
Let's process the nodes file a bit, creating a new file with prefixes as categories.

In [ ]:
node_property_for_color = 'prefix'
node_colors_file_name = node_path + ".colors"
with open(node_path, 'r') as node_file:
    with open(node_colors_file_name, 'w') as node_colors_file:
        header = "id\tprefix\n"
        node_colors_file.write(header)
        node_file.readline()
        for line in node_file:
            splitline=(line.rstrip()).split('\t')
            id = splitline[0]
            prefix = (id.split(':'))[0]
            outline = f"{id}\t{prefix}\n"
            node_colors_file.write(outline)

In [ ]:
outstring = f"""
graph_data:
  graph:
    directed: {directed}
    node_path: {node_path}
    edge_path: {edge_path}
    verbose: True
    nodes_column: 'id'
    node_list_node_types_column: 'category'
    default_node_type: 'biolink:NamedThing'
    sources_column: 'subject'
    destinations_column: 'object'
    default_edge_type: 'biolink:related_to'
  pos_validation:
    edge_path: {pos_valid_edge_path}
  neg_training:
    edge_path: {neg_train_edge_path}
  neg_validation:
    edge_path: {neg_valid_edge_path}

embeddings:
  embedding_file_name: {embedding_file_name}
  embedding_history_file_name: {embedding_history_file_name}
  node_embedding_params:
      node_embedding_method_name: {node_embedding_method_name}
      walk_length: {walk_length}
      batch_size: {batch_size}
      window_size: {window_size}
      return_weight: 1.0
      explore_weight: 1.0
      iterations: {iterations}
      use_mirrored_strategy: False

  tsne:
    tsne_file_name: tsne.png
    color_nodes: True
    node_file: {node_colors_file_name}
    node_property_for_color: {node_property_for_color}

classifier:
  edge_method: {edge_method}
  classifiers:
    - type: {classifier_type}
      model:
        outfile: {classifier_model_outfile}
        type: {classifier_model_type}
        parameters:
          random_state: {classifier_model_random_state}
          max_iter: {classifier_model_max_iter}

output_directory: {output_directory}
"""
print(outstring)

In [ ]:
with open(config_filename, "w") as outfile:
    outfile.write(outstring)

In [ ]:
!pwd
!neat run --config $config_filename

In [ ]:
from IPython.display import Image
Image(filename='tsne.png') 